# BDCC project 1 

_[Big Data and Cloud Computing](http://www.dcc.fc.up.pt/~edrdo/aulas/bdcc), DCC/FCUP_


## Code necessary to run from the command line 

In [7]:
if __name__ == "__main__" :
    # This block is required to run the program from the command line
    # in interface with a single Spark instance
    from pyspark import SparkContext
    from pyspark.sql import SparkSession
    
    spark = SparkSession\
        .builder\
        .appName("BDCCp1")\
        .master("local[*]")\
        .getOrCreate()
    sc = spark.sparkContext
    sc.setLogLevel("WARN")

## Provided code - auxilliary functions

__You should not need to edit these.__

#### loadMovieLensData

In [1]:
from pyspark.sql import functions as F

def readCSV(file, debug=False):
    if debug:
      print('Reading ' + file)
    return spark.read.csv(file, inferSchema=True, header=True)

def readParquet(file, debug=False): 
    if debug:
       print('Reading ' + file)
    return spark.read.parquet(file)

def loadMovieLensData(path, format='parquet', debug=False):
    if format == 'parquet':
       movies = readParquet(path +'/movies.parquet', debug)
       ratings = readParquet(path +'/ratings.parquet', debug)
       tags = readParquet(path +'/tags.parquet', debug)
    else:
       movies = readCSV(path +'/movies.csv', debug)
       ratings = readCSV(path +'/ratings.csv', debug)
       tags = readCSV(path +'/tags.csv', debug)
    
    tags = tags.withColumn('tagl', F.explode(F.split(F.lower(F.col('tag')),'[ \*\+\&\/\%\-\$\#\'\)\(\[\[\],.!?;:\t\n"]+')))\
            .drop('tag')\
            .withColumnRenamed('tagl','tag')
    if (debug):
        print('> movies')
        movies.printSchema()
        movies.show()
        print('> ratings')
        ratings.printSchema()
        ratings.show()
        print('> tags')
        tags.printSchema()
        tags.show()
    return (movies, ratings, tags)

#### writeCSV / writeParquet (use them to write a data frame to CSV or Parquet format)

In [2]:
def writeCSV(df, path): 
    df.write.csv(path, header=True, mode='overwrite')

def writeParquet(df,path):
    df.write.parquet(path, mode='overwrite')


#### createTagListDF

In [3]:
def createTagListDF(csvTagList):
    return spark.createDataFrame([ (t,) for t in csvTagList.split(' ')], ['tag'])

#### Definition of functions available only in Spark 2.4 (GCP Spark instances run Spark 2.3) 

In [4]:
from pyspark.sql import functions as F
from pyspark.sql.types import ArrayType,IntegerType

# Define F.array_intersect if not defined (Spark version < 2.4)
if not hasattr(F,'array_intersect'):
  F.array_intersect = spark.udf\
    .register('array_intersect', 
       lambda x,y: list(set(x) & set(y)), ArrayType(IntegerType()))

# Define F.array_union if not defined (Spark version < 2.4)
if not hasattr(F,'array_union'):
  F.array_union = spark.udf\
    .register('array_union', 
       lambda x,y: list(set(x) | set(y)), ArrayType(IntegerType()))

## Functions to define 

__This is the section that will be evaluated.__

__Include your code for the various functions required in the assigment below.__

__You may include other auxilliary functions required for computation here
but NOT test code (see below).__



#### tfidfTags

In [5]:
from pyspark.sql import functions as F

def tfidfTags(tags, debug=False):
    f = tags\
        .groupBy('tag', 'movieId')\
        .agg(F.count('movieId').alias('f'))
    if debug:
        print('>> Step 1 :: Compute number of times tag has been used in association to movieId')
        f.show()
    
    f_max = f.groupBy('movieId')\
                .agg(F.max('f')\
                     .alias('f_max')\
                )
    f_f_max = f.join(f_max, 'movieId')
    if debug:
        print('>> Step 2 :: Compute maximum absolute frequence of any tag user for movieId')
        f_f_max.show()
    
    tf = f_f_max\
            .withColumn('TF', f_f_max.f / f_f_max.f_max)
    if debug:
        print('>> Step 3 :: TF value of tag for movieId')
        tf.show()
    
    n = tags\
        .drop('userId')\
        .distinct()\
        .groupBy('tag')\
        .agg(F.count('movieId')\
             .alias('n')\
        )
    tf_n = tf.join(n, 'tag')
    if debug:
        print('>> Step 4 :: Join with the number of movies tagged with tag at least once')
        tf_n.show()

    N = tags.select('movieId').distinct().count()
    idf = tf_n\
            .withColumn('IDF',  F.log2(N / tf_n.n))
    if debug:
        print('>> Step 5 :: IDF value of tag considering all tagged movies')
        idf.show()
    
    tfidf = idf\
                .withColumn('TF_IDF',idf.TF * idf.IDF)
    if debug:
        print('>> Step 6 :: TF-IDF value of tag for movieId')
        tfidf.show()
        
    return tfidf
                

#### recommendByTag

In [6]:
from pyspark.sql import functions as F

def recommendByTag(singleTag, TFIDF_tags, movies, min_fmax=10, numberOfResults=10, debug=False):
    filt_tags = TFIDF_tags\
                        .filter((TFIDF_tags.tag == singleTag) &\
                                (TFIDF_tags.f_max >= min_fmax))\
                        .drop('tag', 'f', 'f_max', 'n', 'TF', 'IDF')
    if debug:
        print('>> Step 1 :: TFIDF of single tag & Filtered by *min_fmax*')
        filt_tags.show()

    tags_movie = filt_tags.join(movies, 'movieId')
    if debug:
        print('>> Step 2 :: Join with the corresponding movie')
        tags_movie.show()

    rm_tag = tags_movie\
                .limit(numberOfResults)\
                .orderBy(['TF_IDF', 'title'], ascending=[0, 1])\
                .select('movieId', 'title', 'TF_IDF')
    if debug:
        print('>> Step 3 :: Limit to *numberOfResults* ordered results')
        rm_tag.show()

    return rm_tag

#### recommendByTags

In [7]:
from pyspark.sql import functions as F

# Can it be done by using previous function recommendByTag?
# Even if possible is more computationally heavy
def recommendByTags(searchTags, TFIDF_tags, movies, min_fmax=10, numberOfResults=10, debug=False):
    searchTagsDF = createTagListDF(searchTags)
    if debug:
        print('>> Step 1 :: Search tags DF: ' + searchTags)
        searchTagsDF.show()

    filt_tags = TFIDF_tags\
                    .join(searchTagsDF, 'tag')\
                    .filter(F.col('f_max') >= min_fmax)
    if debug:
        print('>> Step 2 :: TFIDF of given tags & filter by *min_fmax*')
        filt_tags.show()

    sum_tfidf = filt_tags\
                    .groupBy('movieID')\
                    .agg(F.sum('TF_IDF')\
                        .alias('SUM_TF_IDF')\
                    )
    if debug:
        print('>> Step 3 :: Sum of TF_IDF on same movies')
        sum_tfidf.show()

    tags_movie = sum_tfidf\
                    .join(movies, 'movieId')\
                    .limit(numberOfResults)\
                    .orderBy(['SUM_TF_IDF', 'title'], ascending=[0, 1])\
                    .select('movieId', 'title', 'SUM_TF_IDF')
    if debug:
        print('>> Step 4 :: Join with the corresponding movie & limit to *numberOfResults* ordered results')
        tags_movie.show()

    return tags_movie

#### jiMovieSimilarity

In [34]:
from pyspark.sql import functions as F

def jiMovieSimilarity(ratings, minRatings=10, debug=False):
    liked_ratings = ratings\
                        .filter(ratings.rating >= 4.0)
    if debug:
        print('>> Step 1 :: Filter ratings for liked movies')
        liked_ratings.show()

    m1_fm1 = liked_ratings\
                .withColumnRenamed('movieId', 'm1')\
                .groupBy('m1')\
                .agg(F.collect_set(liked_ratings.userId)\
                      .alias('fm1')\
                    )
    if debug:
        print('>> Step 2 :: m1 & Set of users that like a movie (fm1)')
        m1_fm1.show()

    m2_fm2 = m1_fm1\
                .withColumnRenamed('m1', 'm2')\
                .withColumnRenamed('fm1', 'fm2')

    cross_prod = m1_fm1\
                .crossJoin(m2_fm2)\
                .filter(m1_fm1.m1 < m2_fm2.m2)
    if debug:
        print('>> Step 3 :: Crossing different movies and the sets of users that liked each one of them')
        cross_prod.show()

    m1_m2_i_u = cross_prod\
                .withColumn('i', F.size(\
                             F.array_intersect(cross_prod.fm1,\
                                               cross_prod.fm2)\
                                       )\
                           )\
                .withColumn('u', F.size(\
                           F.array_union(cross_prod.fm1,\
                                         cross_prod.fm2)\
                                       )\
                           )\
                .drop('fm1', 'fm2')
    if debug:
        print('>> Step 4 :: # Users that liked m1 and m2 (i) & # Users that liked m1 or m2 (u)')
        m1_m2_i_u.show()

    result = m1_m2_i_u\
                .withColumn('JI', m1_m2_i_u.i / m1_m2_i_u.u)
    if debug:
        print('>> Step 5 :: Computed JI out of i & u')
        result.show()

    return result

#### recommendBySimilarity

In [54]:
def recommendBySimilarity(movieId, movies, jiForMovies, numberOfResults=10, debug=False):
    m1_movieId = jiForMovies\
                    .filter(jiForMovies.m1 == movieId)\
                    .drop('m1', 'i', 'u')
    if debug:
        print('>> Step 1 :: Filter ji where m1 is the given movie Id')
        m1_movieId.show()

    m2_movieId = jiForMovies\
                    .filter(jiForMovies.m2 == movieId)\
                    .drop('m2', 'i', 'u')
    if debug:
        print('>> Step 2 :: Filter ji where m2 is the given movie Id')
        m2_movieId.show()

    ji_movieId = m1_movieId\
                    .withColumnRenamed('m2', 'movieId')\
                    .union(\
                         m2_movieId.withColumnRenamed('m1', 'movieId')\
                          )
    if debug:
        print('>> Step 3 :: Union of the two DFs presented before')
        ji_movieId.show()

    result = ji_movieId\
                    .join(movies, 'movieId')\
                    .select('movieId', 'title', 'JI')\
                    .orderBy('JI', ascending=False)\
                    .limit(numberOfResults)
    if debug:
        print('>> Step 4 :: Join with the respective movies and order results')
        result.show()
        
    return result

# Specify input data set and load it

In [8]:
# Load data
bucket = 'gs://bdcc_up201503784_311' # Ed's bucket 
path = '/p1/data/'
dataset = 'tiny2'
fullPath = bucket + path + dataset

(movies, ratings, tags) = \
  loadMovieLensData(fullPath, format='csv', debug=True)

Reading gs://bdcc_up201503784_311/p1/data/tiny2/movies.csv
Reading gs://bdcc_up201503784_311/p1/data/tiny2/ratings.csv
Reading gs://bdcc_up201503784_311/p1/data/tiny2/tags.csv
> movies
root
 |-- movieId: integer (nullable = true)
 |-- title: string (nullable = true)

+-------+--------------------+
|movieId|               title|
+-------+--------------------+
|      1|    Toy Story (1995)|
|      2|      Jumanji (1995)|
|      3|Grumpier Old Men ...|
|      4|Waiting to Exhale...|
|      5|Father of the Bri...|
|      6|         Heat (1995)|
|      7|      Sabrina (1995)|
|      8| Tom and Huck (1995)|
|      9| Sudden Death (1995)|
|     10|    GoldenEye (1995)|
|     11|American Presiden...|
|     12|Dracula: Dead and...|
|     13|        Balto (1995)|
|     14|        Nixon (1995)|
|     15|Cutthroat Island ...|
|     16|       Casino (1995)|
|     17|Sense and Sensibi...|
|     18|   Four Rooms (1995)|
|     19|Ace Ventura: When...|
|     20|  Money Train (1995)|
+-------+----------

##  Test code 

__Include test code below that you may need here.__

__The initial contents are only meant as an example.__

__This section will NOT be evaluated.__

In [71]:
# Get TF-IDF for tags
tfidf = tfidfTags(tags, debug=True)

# tfidf.cache()
# tfidf.orderBy(['movieId','TF_IDF'],ascending=[1,0]).show()
tfidf.orderBy(['f','TF_IDF','movieId','tag'],ascending=[0,0,1,1]).show()



>> Step 1 :: Compute number of times tag has been used in association to movieId
+----------+-------+---+
|       tag|movieId|  f|
+----------+-------+---+
|      game|      2|  2|
|     moldy|      3|  1|
|alcoholism|     25|  1|
|     mafia|     16|  1|
|  williams|      2|  1|
| president|     14|  1|
|    austen|     17|  1|
|     robin|      2|  1|
|     pixar|      1|  2|
|     magic|      2|  1|
|   fantasy|      2|  1|
|    remake|      5|  1|
|      jane|     17|  1|
|       fun|      1|  1|
|  politics|     11|  1|
| pregnancy|      5|  1|
|    remake|      7|  1|
|    serial|     22|  1|
| president|     11|  1|
|     board|      2|  1|
+----------+-------+---+
only showing top 20 rows

>> Step 2 :: Compute maximum absolute frequence of any tag user for movieId
+-------+----------+---+-----+
|movieId|       tag|  f|f_max|
+-------+----------+---+-----+
|      2|      game|  2|    2|
|      3|     moldy|  1|    1|
|     25|alcoholism|  1|    1|
|     16|     mafia|  1|    1|


In [69]:
# Recommend by tag 

rm = recommendByTag('pixar', tfidf, movies, min_fmax=1, debug=True)
rm.show()

rm = recommendByTag('politics', tfidf, movies, min_fmax=1)
rm.show()


rm = recommendByTag('remake', tfidf, movies, min_fmax=1)
rm.show()




>> Step 1 :: TFIDF of single tag & Filtered by *min_fmax*
+-------+------------------+
|movieId|            TF_IDF|
+-------+------------------+
|      1|3.5849625007211565|
+-------+------------------+

>> Step 2 :: Join with the corresponding movie
+-------+------------------+----------------+
|movieId|            TF_IDF|           title|
+-------+------------------+----------------+
|      1|3.5849625007211565|Toy Story (1995)|
+-------+------------------+----------------+

>> Step 3 :: Limit to *numberOfResults* ordered results
+-------+----------------+------------------+
|movieId|           title|            TF_IDF|
+-------+----------------+------------------+
|      1|Toy Story (1995)|3.5849625007211565|
+-------+----------------+------------------+

+-------+----------------+------------------+
|movieId|           title|            TF_IDF|
+-------+----------------+------------------+
|      1|Toy Story (1995)|3.5849625007211565|
+-------+----------------+------------------+



In [59]:
# Recommend by Tags

rm = recommendByTags('robin williams remake', tfidf, movies, min_fmax=1, debug=True)
rm.show()

rm = recommendByTags('pixar fantasy', tfidf, movies, min_fmax=1)
rm.show()

rm = recommendByTags('serial killer', tfidf, movies, min_fmax=1)
rm.show()

#rm = recommendByTags('hitchcock birds', tfidf, movies, numberOfResults=10)
#rm.show()




>> Step 1 :: Search tags DF: robin williams remake
+--------+
|     tag|
+--------+
|   robin|
|williams|
|  remake|
+--------+

>> Step 2 :: TFIDF of given tags & filter by *min_fmax*
+--------+-------+---+-----+---+---+------------------+------------------+
|     tag|movieId|  f|f_max| TF|  n|               IDF|            TF_IDF|
+--------+-------+---+-----+---+---+------------------+------------------+
|   robin|      2|  1|    2|0.5|  1|3.5849625007211565|1.7924812503605783|
|williams|      2|  1|    2|0.5|  1|3.5849625007211565|1.7924812503605783|
|  remake|      5|  1|    1|1.0|  2| 2.584962500721156| 2.584962500721156|
|  remake|      7|  1|    1|1.0|  2| 2.584962500721156| 2.584962500721156|
+--------+-------+---+-----+---+---+------------------+------------------+

>> Step 3 :: Sum of TF_IDF
+-------+------------------+
|movieID|        SUM_TF_IDF|
+-------+------------------+
|      5| 2.584962500721156|
|      7| 2.584962500721156|
|      2|3.5849625007211565|
+-------+----

In [39]:
jiM = jiMovieSimilarity(ratings, debug=True)

#jiM.orderBy(['JI','m1','m2'], ascending=[0,1,1]).show()
jiM.orderBy(['i','JI','m1','m2'], ascending=[0,0,1,1]).show()




>> Step 1 :: Filter ratings for liked movies
+-------+------+------+
|movieId|userId|rating|
+-------+------+------+
|      1|     1|   4.0|
|      3|     1|   4.0|
|      6|     1|   4.0|
|      1|     5|   4.0|
|     21|     5|   4.0|
|      2|     6|   4.0|
|      3|     6|   5.0|
|      5|     6|   5.0|
|      6|     6|   4.0|
|      7|     6|   4.0|
|     11|     6|   4.0|
|     15|     6|   4.0|
|     16|     6|   4.0|
|     17|     6|   4.0|
|     22|     6|   5.0|
|     24|     6|   4.0|
|      1|     7|   4.5|
|      2|     8|   4.0|
|     11|     8|   4.0|
|     21|     8|   4.0|
+-------+------+------+
only showing top 20 rows

>> Step 2 :: m1 & Set of users that like a movie (fm1)
+---+--------------------+
| m1|                 fm1|
+---+--------------------+
| 12|     [380, 351, 276]|
| 22|[353, 99, 6, 42, ...|
|  1|[610, 277, 460, 2...|
| 13|           [20, 304]|
|  6|[610, 437, 577, 4...|
| 16|[610, 437, 483, 5...|
|  3|[269, 51, 102, 1,...|
| 20|               [464]|
|

In [56]:
jiM.cache()

# Pulp Fiction
#sm = recommendBySimilarity(296, movies, jiM)
#sm.show()

# Fight club
#sm = recommendBySimilarity(2959, movies, jiM)
#sm.show()
    
# Shrek
#sm = recommendBySimilarity(4306, movies, jiM)
#sm.show()

# Toy Story
sm = recommendBySimilarity(1, movies, jiM)
sm.show()

# Heat
sm = recommendBySimilarity(6, movies, jiM)
sm.show()

# Leaving Las Vegas
sm = recommendBySimilarity(25, movies, jiM)
sm.show()

+-------+--------------------+-------------------+
|movieId|               title|                 JI|
+-------+--------------------+-------------------+
|      6|         Heat (1995)|0.14285714285714285|
|     21|   Get Shorty (1995)|0.12941176470588237|
|      2|      Jumanji (1995)|0.11931818181818182|
|     25|Leaving Las Vegas...| 0.1111111111111111|
|     17|Sense and Sensibi...|0.10650887573964497|
|     10|    GoldenEye (1995)|0.10160427807486631|
|     11|American Presiden...|0.09826589595375723|
|     16|       Casino (1995)| 0.0967741935483871|
|      3|Grumpier Old Men ...|0.07142857142857142|
|     19|Ace Ventura: When...|0.06451612903225806|
+-------+--------------------+-------------------+

+-------+--------------------+-------------------+
|movieId|               title|                 JI|
+-------+--------------------+-------------------+
|     16|       Casino (1995)|             0.3125|
|     25|Leaving Las Vegas...|0.21739130434782608|
|      1|    Toy Story (1995)|